In [10]:
import os
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig
from azureml.core.dataset import Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
# from azureml.core.conda_dependencies import CondaDependencies
import json

from dotenv import load_dotenv
load_dotenv()

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.49.0


### Prihlasenie do azure

In [11]:
with open('../azure/config_siemens.json', 'r') as f:
    conf = json.load(f)
    ws = Workspace.from_config(path='config_siemens.json', auth=InteractiveLoginAuthentication(tenant_id=conf['directory_id']))

### Nacitanie datasetu

In [12]:
# Dataset.get_all(workspace=ws)

In [13]:
dataset = Dataset.get_by_name(workspace=ws, name='povinec-vs')
# dataset

## Experimenty

In [14]:
config = ScriptRunConfig(
    source_directory='../src/',
    # script='train.py',
    script='train_correction.py',
    # script='correction_sweep.py',
    # script='evaluation.py',
    arguments=[
        '--data-path', dataset.as_mount(), 
        '--wandb-key', os.getenv('WANDB'),
        '--use-wandb', True,
        '--name', 'correctionloss_128imgs_volumetric_seq',
        # '--model-path', './models/pretrained_dice_30.pt'
    ], 
    compute_target='V100x1-xpovinec'
    # compute_target='xslimak-V100'
)

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


### Vytvorenie experimentu

In [15]:
experiment = Experiment(workspace=ws, name='xpovinec-VS')

### Nastavenie env

In [16]:
env = Environment.get(workspace=ws, name='xpovinec-env')
config.run_config.environment = env

### Spustenie experimentu

In [17]:
run = experiment.submit(config)

In [18]:
# toto zisti, kde bezi a vrati to stranku
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/runs/xpovinec-VS_1714383237_964b1b75?wsid=/subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourcegroups/a0047-STUFIIT-ML01/workspaces/a0047stufiitml01&tid=5dbf1add-202a-4b8d-815b-bf0fb024e033
